## Recuperación puntual (PITR) para Amazon DynamoDB

Amazon DynamoDB le permite realizar backups continuos de los datos de las tablas mediante la recuperación puntual (PITR). Cuando habilita PITR, DynamoDB realiza una copia de seguridad automática de los datos de la tabla con una granularidad por segundo, de manera que puede restaurar a cualquier segundo dado en los 35 días anteriores.

PITR le ayuda a protegerse contra escrituras y eliminaciones accidentales. Por ejemplo, si un script de prueba escribe accidentalmente en una tabla de DynamoDB de producción o si alguien realiza una llamada "DeleteItem" por error, PITR le protege. 

Con PITR, puede realizar copias de seguridad de tablas con cientos de terabytes de datos, sin que ello afecte al rendimiento o la disponibilidad de sus aplicaciones de producción. También puede recuperar tablas de DynamoDB habilitadas para PITR que se hayan eliminado en los 35 días anteriores y restaurar tablas al estado en el que se encontraban justo antes de su eliminación.

Los precios de las copias de seguridad continuas se detallan en las páginas de precios de DynamoDB. Los precios varían según la región y se basan en el tamaño actual de la tabla y los índices. Por ejemplo, en el este de Estados Unidos (norte de Virginia) se pagan 0,20 dólares por GB en función del tamaño de los datos y de todos los índices secundarios locales.

Hay que tener en cuenta algunas cosas:

* PITR funciona con tablas encriptadas.

* Si desactivas PITR y lo vuelves a activar más tarde, restableces la hora de inicio a partir de la cual puedes recuperarte.

* Al igual que las copias de seguridad bajo demanda, la activación de esta función no afecta al rendimiento ni a la disponibilidad.

* La configuración de streaming, la configuración de Time To Live, la configuración de PITR, las etiquetas, las alarmas de Amazon CloudWatch y las políticas de escalado automático no se copian en la tabla restaurada.

In [45]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime
import time

In [23]:
dt=DynamoTable()
try:
    dt.select_table('VerySuperImportantTable')
    print(dt)
except:
    dt.create_table(
        table_name='VerySuperImportantTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S'
    )

Table created successfully!


In [42]:
df = pd.read_csv('data.csv')
dt.batch_pandas(df)

Imaginemos un escenario en el que tengo muchos perfiles de usuario antiguos que quiero eliminar de mi tabla de DynamoDB. En lugar de eliminarlos directamente, puedo habilitar PITR y, a continuación, eliminar los perfiles de usuario. Si me doy cuenta de que he cometido un error, puedo restaurar la tabla a un momento anterior y recuperar los perfiles de usuario eliminados.

In [36]:
# Enable PITR
dynamo_client = boto3.client('dynamodb', region_name='us-east-1')
dynamo_client.update_continuous_backups(
    TableName='VerySuperImportantTable',
    PointInTimeRecoverySpecification={
        'PointInTimeRecoveryEnabled': True
    }
)

{'ContinuousBackupsDescription': {'ContinuousBackupsStatus': 'ENABLED',
  'PointInTimeRecoveryDescription': {'PointInTimeRecoveryStatus': 'ENABLED',
   'EarliestRestorableDateTime': datetime.datetime(2023, 6, 27, 13, 20, 41, tzinfo=tzlocal()),
   'LatestRestorableDateTime': datetime.datetime(2023, 6, 27, 13, 20, 41, tzinfo=tzlocal())}},
 'ResponseMetadata': {'RequestId': 'NK1NA3282NRGK9V3KJ6V4QRMI7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 27 Jun 2023 16:20:41 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '227',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'NK1NA3282NRGK9V3KJ6V4QRMI7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3161676638'},
  'RetryAttempts': 0}}

In [ ]:
dt.status_pitr

In [46]:
now = datetime.now()
time.sleep(3)
items = dt.table.scan(
    FilterExpression="GSI1SK >= :date",
    ExpressionAttributeValues={":date": "1900-12-10"},
)['Items']

In [47]:
print("Deleting {} Items! Dangerous.".format(len(items)))
with dt.table.batch_writer() as batch:
    for item in items:
        itm = [item['PK'], item['SK']]
        batch.delete_item(Key={'PK': itm[0], 'SK': itm[1]})

Deleting 10 Items! Dangerous.


In [48]:
all_items = dt.table.scan()['Items']
print("There are {} items in the table.".format(len(all_items)))

There are 0 items in the table.


In [53]:
now.strftime("%Y-%m-%dT%H:%M:%S%mZ")

'2023-06-27T13:38:2706Z'

In [54]:
# Restore the table
target_table = 'VerySuperImportantTable-restored'
dynamo_client.restore_table_to_point_in_time(
    TargetTableName=target_table,
    SourceTableName='VerySuperImportantTable',
    UseLatestRestorableTime=True
)

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'SK',
    'AttributeType': 'S'},
   {'AttributeName': 'PK', 'AttributeType': 'S'}],
  'TableName': 'VerySuperImportantTable-restored',
  'KeySchema': [{'AttributeName': 'PK', 'KeyType': 'HASH'},
   {'AttributeName': 'SK', 'KeyType': 'RANGE'}],
  'TableStatus': 'CREATING',
  'CreationDateTime': datetime.datetime(2023, 6, 27, 13, 42, 14, 450000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:266147702039:table/VerySuperImportantTable-restored',
  'TableId': '4af7258e-5274-4a7c-8f62-f5022824d3f6',
  'BillingModeSummary': {'BillingMode': 'PROVISIONED'},
  'RestoreSummary': {'SourceTableArn': 'arn:aws:dynamodb:us-east-1:266147702039:table/VerySuperImportantTable',
   'RestoreDateTime': datetime.datetime(2023, 6, 27, 13, 37, 14, 541000, tzinfo=tzlocal()),
   'Res

In [55]:
dt.select_table(target_table)
print(dt)

- Table name: VerySuperImportantTable-restored            
- Table arn: arn:aws:dynamodb:us-east-1:266147702039:table/VerySuperImportantTable-restored            
- Table creation: 2023-06-27 13:42:14            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [57]:
all_items = dt.table.scan()['Items']
print("There are {} items in the table.".format(len(all_items)))

There are 10 items in the table.


#### Eliminación de una tabla con la recuperación a un momento dado habilitada

Cuando se elimina una tabla que tiene habilitada la recuperación a un momento dado, DynamoDB crea automáticamente una instantánea de copia de seguridad, denominada system backup (copia de seguridad del sistema) y la mantiene durante 35 días (sin costo adicional). Puede utilizarla para restaurar la tabla eliminada al estado en el que se encontraba junto antes de la eliminación. Todas las copias de seguridad del sistema siguen una convención de nomenclatura estándar de nombre-tabla$DeletedTableBackup.

In [58]:
# Delete dynamodb table
response = dynamo_client.delete_table(
    TableName='VerySuperImportantTable' 
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Table deleted.")

Table deleted.


En la consola de DynamoDB, puede ver las copias de seguridad en la pestaña Backups